# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

## Answer
The data represents 17 campaigns between May 2008 and November 2010.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [29]:
import pandas as pd

In [88]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [73]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



## Answer
* Columns 2 through 7 need to handle the 'unknown' value potentially dropping the data.
* Columns 8 through 10 are categorical.
* Column 11 happens after the call is complete so it can't be treated like a regular column.
* Column 13 a value of 999 needs special handling


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

## Answer
The Business Objective of the Task is to utilize the data at hand to characterize what drives people to subscribe to provide the focus areas for marketing teams.

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [104]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

# Check if the target variable 'y' is present
if 'y' not in df.columns:
    raise KeyError("The target variable 'y' is missing from the dataset.")

# Verify the column names and print the first few rows
print(df.columns)
print(df.head())

# Select bank information features and target variable
bank_info_features = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
target = 'loan_yes'

# Impute missing values for categorical and numeric features
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan']
numeric_features = ['age']

# Imputer for categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')

# Imputer for numeric features
num_imputer = SimpleImputer(strategy='mean')

# Apply imputers
df[categorical_features] = cat_imputer.fit_transform(df[categorical_features])
df[numeric_features] = num_imputer.fit_transform(df[numeric_features])

# One-hot encode the categorical features
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Fit and transform categorical features
encoded_features = encoder.fit_transform(df[categorical_features])

# Convert the encoded features to a DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Concatenate the encoded features with the original DataFrame
df = pd.concat([df[numeric_features], encoded_df], axis=1)

# Verify columns after encoding
print(df.columns)

# Label encode the target variable
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

# Display the final DataFrame
print(df.head())


Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')
   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   m

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [106]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (32950, 27)
X_test shape: (8238, 27)
y_train shape: (32950,)
y_test shape: (8238,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [108]:
# Calculate class distribution in the target variable
class_counts = y_train.value_counts()
print("Class Distribution in Training Set:")
print(class_counts)

# Determine the majority class
majority_class = class_counts.idxmax()
majority_class_count = class_counts.max()
print(f"Majority Class: {majority_class}, Count: {majority_class_count}")

# Calculate baseline accuracy
baseline_accuracy = majority_class_count / len(y_train)
print(f"Baseline Accuracy: {baseline_accuracy:.2f}")

Class Distribution in Training Set:
loan_yes
0    27946
1     5004
Name: count, dtype: int64
Majority Class: 0, Count: 27946
Baseline Accuracy: 0.85


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [122]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

### Problem 9: Score the Model

What is the accuracy of your model?

In [124]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, zero_division=0)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.85
Confusion Matrix:
[[6994    0]
 [1244    0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      6994
           1       0.00      0.00      0.00      1244

    accuracy                           0.85      8238
   macro avg       0.42      0.50      0.46      8238
weighted avg       0.72      0.85      0.78      8238



### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [135]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import time

import os
os.environ["LOKY_MAX_CPU_COUNT"] = '12' # Needed to use this on my ARM Windows machine

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=500),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(random_state=42)
}

results_list = []

# Fit, score and record metrics for each model
for model_name, model in models.items():
    start_time = time.time()
    
    model.fit(X_train_scaled, y_train)
    train_time = time.time() - start_time
    
    train_accuracy = model.score(X_train_scaled, y_train)
    test_accuracy = model.score(X_test_scaled, y_test)
    
    results_list.append({
        "Model": model_name,
        "Train Time": train_time,
        "Train Accuracy": train_accuracy,
        "Test Accuracy": test_accuracy
    })

# Convert results list to DataFrame
results = pd.DataFrame(results_list)

# Display the results DataFrame
print(results)

TypeError: str expected, not int

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

## Answer

### Feature Engineering
I could potentially deep-dive into domain knowledge (perhaps digging into the PDF more) and data analysis. I would likely suggest creating new features or interactions or transforming existing ones. Then, after each change, I would validate the impact of each feature.

### Performance Metrics
* Precision and Recall: Useful if false positives/negatives have different costs.
* F1-Score: Combination of both precision and recall.
* ROC AUC: Evaluates the probability of correct classification across thresholds.

### Hyperparameter Tuning
Use GridSearchCV/RandomizedSearchCV with cross-validation to fine-tune models—this step could likely make significant improvements.

### Conclusion
Of the three - I would likely start with hyperparameter tuning and see how far that improves the models.